## Import Dataset

In [1]:
from teaspoon.ML import load_datasets
mnist = load_datasets.mnist()
mnist

,zero_dim_rtl,zero_dim_ltr,zero_dim_btt,zero_dim_ttb,one_dim_rtl,one_dim_ltr,one_dim_btt,one_dim_ttb,labels
0,"[[23.0, 50.0], [13.0, 20.0]]","[[24.0, 50.0], [11.0, 21.0]]","[[24.0, 50.0], [8.0, 9.0]]","[[23.0, 50.0]]",[],[],[],[],5
1,"[[22.0, 50.0]]","[[22.0, 50.0]]","[[23.0, 50.0]]","[[24.0, 50.0]]","[[0.0, 8.0]]","[[0.0, 8.0]]","[[0.0, 9.0]]","[[0.0, 8.0]]",0
2,"[[22.0, 50.0]]","[[25.0, 50.0]]","[[24.0, 50.0], [15.0, 16.0]]","[[23.0, 50.0], [13.0, 22.0]]",[],[],[],[],4
3,"[[21.0, 50.0]]","[[20.0, 50.0]]","[[24.0, 50.0]]","[[23.0, 50.0]]",[],[],[],[],1
4,"[[20.0, 50.0], [16.0, 18.0]]","[[22.0, 50.0], [14.0, 15.0]]","[[26.0, 50.0]]","[[21.0, 50.0]]","[[0.0, 9.0]]","[[0.0, 12.0]]","[[0.0, 9.0]]","[[0.0, 14.0]]",9
...,...,...,...,...,...,...,...,...,...
59995,"[[23.0, 50.0], [16.0, 17.0]]","[[22.0, 50.0], [16.0, 18.0]]","[[24.0, 50.0]]","[[23.0, 50.0]]","[[0.0, 13.0], [0.0, 7.0]]","[[0.0, 13.0], [0.0, 9.0]]","[[0.0, 16.0], [0.0, 6.0]]","[[0.0, 17.0], [0.0, 5.0]]",8
59996,"[[21.0, 50.0], [17.0, 18.0]]","[[24.0, 50.0], [12.0, 18.0], [9.0, 17.0]]","[[23.0, 50.0]]","[[24.0, 50.0], [7.0, 10.0]]",[],[],[],[],3
59997,"[[24.0, 50.0], [13.0, 18.0]]","[[23.0, 50.0], [12.0, 20.0]]","[[24.0, 50.0]]","[[23.0, 50.0], [5.0, 7.0]]",[],[],[],[],5
59998,"[[23.0, 50.0], [8.0, 21.0]]","[[22.0, 50.0], [11.0, 15.0]]","[[21.0, 50.0]]","[[26.0, 50.0], [8.0, 15.0]]","[[0.0, 17.0]]","[[0.0, 8.0]]","[[0.0, 15.0]]","[[0.0, 10.0]]",6


### Choose Dimension for Timing Test

In [2]:
def train_test_split_sklearn(DgmsFD, labels_col, train_size=.5, seed=12):
    from sklearn.model_selection import train_test_split
    labels = DgmsFD[labels_col]
    training_dgms, testing_dgms = train_test_split(DgmsFD, train_size=train_size, random_state=seed, stratify=labels)
    return training_dgms.reset_index(), testing_dgms.reset_index()

In [3]:
dgms_train, dgms_test = train_test_split_sklearn(mnist, 'labels', train_size = .95)
xdgm0_train = dgms_train['zero_dim_rtl']
xdgm0_test = dgms_test['zero_dim_rtl']
xdgm1_train = dgms_train['one_dim_rtl']
xdgm1_test = dgms_test['one_dim_rtl']
labels_train = dgms_train['labels']
labels_test = dgms_test['labels']

### Load original kernel method function from teaspoon

In [5]:
import numpy as np
import math
from numpy.linalg import norm as lnorm
from math import pi

def KernelMethod(perDgm1, perDgm2, sigma):
    
    L1 = len(perDgm1)
    L2 = len(perDgm2)
    kernel = np.zeros((L2, L1))

    Kernel = 0

    for i in range(0, L1):
        p = perDgm1[i]
        p = np.reshape(p, (2, 1))
        for j in range(0, L2):
            q = perDgm2[j]
            q = np.reshape(q, (2, 1))
            q_bar = np.zeros((2, 1))
            q_bar[0] = q[1]
            q_bar[1] = q[0]
            dist1 = lnorm(p-q)
            dist2 = lnorm(p-q_bar)
            kernel[j, i] = np.exp(-(math.pow(dist1, 2))/(8*sigma)) - \
                np.exp(-(math.pow(dist2, 2))/(8*sigma))
            Kernel = Kernel+kernel[j, i]
    Kernel = Kernel*(1/(8*pi*sigma))

    return Kernel

def heat_kernel_distance(dgm0, dgm1, sigma=.4):
    return np.sqrt(KernelMethod(dgm0, dgm0, sigma) + KernelMethod(dgm1, dgm1, sigma) - 2*KernelMethod(dgm0, dgm1, sigma))

def kernel_features(train, s):
    import time
    import numpy as np
    n_train = len(train)
    X_train_features = np.zeros((n_train, n_train))
    
    start = time.time()
    for i in range(0,n_train):
        for j in range(0,i):
            dgm0 = train[i]
            dgm1 = train[j]
            hka = heat_kernel_distance(dgm0, dgm1, sigma = s) 
            X_train_features[i,j] = hka
            X_train_features[j,i] = hka

    timing = time.time()-start
    return timing, X_train_features

Optimized with Numba

In [84]:
import numpy as np
import math
from numpy.linalg import norm as lnorm
from math import pi
from numba import vectorize
import time

@jit
def optimizedKernelMethod(perDgm1, perDgm2, sigma):
    L1 = len(perDgm1)
    L2 = len(perDgm2)
    kernel = np.zeros((L2, L1))

    Kernel = 0

    for i in range(0, L1):
        p = perDgm1[i]
        p = np.reshape(p, (2, 1))
        for j in range(0, L2):
            q = perDgm2[j]
            q = np.reshape(q, (2, 1))
            q_bar = np.zeros((2, 1))
            q_bar[0] = q[1]
            q_bar[1] = q[0]
            dist1 = lnorm(p-q)
            dist2 = lnorm(p-q_bar)
            kernel[j, i] = np.exp(-(math.pow(dist1, 2))/(8*sigma)) - \
                np.exp(-(math.pow(dist2, 2))/(8*sigma))
            Kernel = Kernel+kernel[j, i]
    Kernel = Kernel*(1/(8*pi*sigma))

    return Kernel

@jit
def optimized_heat_kernel_distance(dgm0, dgm1, sigma=.4):
    return np.sqrt(optimizedKernelMethod(dgm0, dgm0, sigma) + optimizedKernelMethod(dgm1, dgm1, sigma) - 2*optimizedKernelMethod(dgm0, dgm1, sigma))

@jit
def optimized_kernel_features(train, s):
    n_train = len(train)
    X_train_features = np.zeros((n_train, n_train))
    
    start = time.time()
    for i in range(0,n_train):
        for j in range(0,i):
            dgm0 = train[i]
            dgm1 = train[j]
            hka = optimized_heat_kernel_distance(dgm0, dgm1, sigma = s) 
            X_train_features[i,j] = hka
            X_train_features[j,i] = hka

    timing = time.time()-start
    return timing, X_train_features

/var/folders/p6/pn26dbmx4jj67d6nh7m6557h0000gn/T/ipykernel_8952/1848828777.py:9: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def optimizedKernelMethod(perDgm1, perDgm2, sigma):
/var/folders/p6/pn26dbmx4jj67d6nh7m6557h0000gn/T/ipykernel_8952/1848828777.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def optimized_heat_kernel_distance(dgm0, dgm1, sigma=.4):

In [114]:
import numpy as np
import math
from numpy.linalg import norm as lnorm
from math import pi
from numba import vectorize
import time
from numba import prange

@vectorize(nopython=True)
def parallel_optimized_kernel_features(train, s):
    n_train = len(train)
    X_train_features = np.zeros((n_train, n_train))
    
    start = time.time()
    for i in range(0,n_train):
        for j in range(0,i):
            dgm0 = train[i]
            dgm1 = train[j]
            hka = optimized_heat_kernel_distance(dgm0, dgm1, sigma = s) 
            X_train_features[i,j] = hka
            X_train_features[j,i] = hka

    timing = time.time()-start
    return timing, X_train_features

In [130]:
from numba import guvectorize
@guvectorize(["void(float64[:,:], float64[:], float64, float64[:,:])",],"(m,n),(p),()->(p,p)", target='parallel')
def numba_kernel_features_train(train, dummy, s, result):
    n_train = len(dummy)
    for i in range(n_train):
        for j in range(i):
            dgm0 = train[train[:,0]==i,1:3]
            dgm1 = train[train[:,0]==j,1:3]
            kSigma0 = 0
            kSigma1 = 0
            kSigma2 = 0
            sigma = s
            for k in range(dgm0.shape[0]):
                p = dgm0[k,0:2]
                for l in range(dgm0.shape[0]):
                    q = dgm0[l,0:2]
                    qc = dgm0[l, 1::-1]
                    pq = (p[0] - q[0])**2 + (p[1] - q[1])**2
                    pqc = (p[0] - qc[0])**2 + (p[1] - qc[1])**2
                    kSigma0 += math.exp(-( pq) / (8 * sigma)) - math.exp(-(pqc) / (8 * sigma))
            for k in range(dgm1.shape[0]):
                p = dgm1[k,0:2]
                for l in range(dgm1.shape[0]):
                    q = dgm1[l,0:2]
                    qc = dgm1[l, 1::-1]
                    pq = (p[0] - q[0])**2 + (p[1] - q[1])**2
                    pqc = (p[0] - qc[0])**2 + (p[1] - qc[1])**2
                    kSigma1 += math.exp(-( pq) / (8 * sigma)) - math.exp(-(pqc) / (8 * sigma))
            for k in range(dgm0.shape[0]):
                p = dgm0[k,0:2]
                for l in range(dgm1.shape[0]):
                    q = dgm1[l,0:2]
                    qc = dgm1[l, 1::-1]
                    pq = (p[0] - q[0])**2 + (p[1] - q[1])**2
                    pqc = (p[0] - qc[0])**2 + (p[1] - qc[1])**2
                    kSigma2 += math.exp(-( pq) / (8 * sigma)) - math.exp(-(pqc) / (8 * sigma))

            kSigma0 = kSigma0/(8 * np.pi * sigma)
            kSigma1 = kSigma1/(8 * np.pi * sigma)
            kSigma2 = kSigma2/(8 * np.pi * sigma)
            result[i,j] = math.sqrt(kSigma1 + kSigma0-2*kSigma2)
            result[j,i] = math.sqrt(kSigma1 + kSigma0-2*kSigma2)

In [129]:
def reshape_persistence_diagrams(dgm):
    dgm_reshape = np.array([])
    n = len(dgm)
    for i in range(0,n):
        t = np.repeat(i, len(dgm[i]))
        t = t.reshape(len(dgm[i]),1)
        t1 = np.concatenate((t,dgm[i]),1)
        if i == 0:
            dgm_reshape = t1
        else:
            dgm_reshape = np.append(dgm_reshape, t1, 0)
    return dgm_reshape

### Loop for random samples and timing

In [134]:

train_test = xdgm0_train[0:100]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
dummy_train = np.arange(len(train_test), dtype=np.float64)
train = reshape_persistence_diagrams(train_test)
%timeit numba_kernel_features_train(train, dummy_train, .3)

366 ms ± 9.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
29.8 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.47 ms ± 8.76 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [137]:
train_test = xdgm0_train[0:200]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
dummy_train = np.arange(len(train_test), dtype=np.float64)
train = reshape_persistence_diagrams(train_test)
%timeit numba_kernel_features_train(train, dummy_train, .3)

1.45 s ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
120 ms ± 416 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
22.1 ms ± 604 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [96]:
train_test = xdgm0_train[0:300]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

3.24 s ± 41.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
272 ms ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
263 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [97]:
train_test = xdgm0_train[0:400]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

5.79 s ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
468 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
466 ms ± 774 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
train_test = xdgm0_train[0:500]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

9.06 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
731 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
729 ms ± 7.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
train_test = xdgm0_train[0:750]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

20.1 s ± 152 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.71 s ± 143 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.66 s ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [100]:
train_test = xdgm0_train[0:1000]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

36.9 s ± 773 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.98 s ± 27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.97 s ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
train_test = xdgm0_train[0:1250]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

55.4 s ± 524 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.67 s ± 34.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.68 s ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
train_test = xdgm0_train[0:1500]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

1min 20s ± 761 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.78 s ± 81.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.81 s ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
train_test = xdgm0_train[0:2000]
%timeit timing, X_train_features = kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = optimized_kernel_features(np.array(train_test), s = .3)
%timeit timing, X_train_features = parallel_optimized_kernel_features(np.array(train_test), s = .3)

KeyboardInterrupt: 